In [148]:
import boto3
import datetime
import os
import requests
import json
from PIL import Image
from io import BytesIO
import qrcode

def generate_s3_filename(extension="png"):
    """
    Genera un nombre de archivo basado en el timestamp actual.
    
    Args:
    - extension (str): extensión del archivo. Por defecto es "jpg".
    
    Returns:
    - str: nombre del archivo generado.
    """
    # Obtiene el timestamp actual y lo formatea
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{timestamp}.{extension}"
    return filename

def load_api_key(json_file='stability-ai-api-key.json'):
    # Abre y lee el archivo JSON
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Extrae el valor asociado a la llave "key"
    return data.get("key", None)

def generate_image(api_key, input_prompt):
    prompt = ""
    # prompt = "an aerial isometric view of a city, "
    prompt += input_prompt
    response = requests.post('https://clipdrop-api.co/text-to-image/v1',
        files = {
            'prompt': (None, prompt, 'text/plain')
        },
        headers = {'x-api-key': api_key}
    )
    if response.status_code == 200:
        return response
    else:
        return None

In [149]:
api_key = load_api_key()
prompt = "Vista area isometrica de una ciudad con edificios y colores estilo Cyberpunk"
response = generate_image(api_key, input_prompt=prompt)

In [150]:
gen_image = Image.open(BytesIO(response.content))

# Convirtiendo la imagen a bytes para subirla a S3
buffer = BytesIO()
gen_image.save(buffer, 'png')
buffer.seek(0)

s3 = boto3.client('s3')
bucket_name = 'stable-diffusion-city-images'
s3_file_name = generate_s3_filename()

In [151]:
s3.put_object(Bucket=bucket_name, Key=s3_file_name, Body=buffer)

{'ResponseMetadata': {'RequestId': 'M5C2A226Z7E10JH5',
  'HostId': 'd40kXUTn2Z2x2bV033RW+WdtiD2rwCjECKRPGl3WHtds5NZJEHO21S7T8qYcQ0O+PzvhcDjM2yk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'd40kXUTn2Z2x2bV033RW+WdtiD2rwCjECKRPGl3WHtds5NZJEHO21S7T8qYcQ0O+PzvhcDjM2yk=',
   'x-amz-request-id': 'M5C2A226Z7E10JH5',
   'date': 'Thu, 02 Nov 2023 19:14:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c19e1cdbbebdee97e9a8fdb7b90192be"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c19e1cdbbebdee97e9a8fdb7b90192be"',
 'ServerSideEncryption': 'AES256'}

In [152]:
image_url = s3.generate_presigned_url('get_object',
                                Params={'Bucket': bucket_name, 'Key': s3_file_name},
                                ExpiresIn=3600)  # URL válida por 1 hora

In [153]:
# Genera el código QR con la URL
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(image_url)
qr.make(fit=True)
qr_img = qr.make_image(fill_color="black", back_color="white")

# Dimensiones deseadas para el QR (por ejemplo, 100x100 píxeles)
desired_size = (150, 150)

# Redimensiona el código QR
qr_img_resized = qr_img.resize(desired_size)

# Calcula la posición para colocar el QR redimensionado en la esquina superior derecha
margin = 10
x_position = gen_image.width - qr_img_resized.width - margin
y_position = margin

image = gen_image.copy()

# Superpone el QR redimensionado en la imagen
image.paste(qr_img_resized, (x_position, y_position))

# Muestra la imagen resultante
image.show()
